# Beecomender Model Implementation

This Jupyter Notebook has been created to provide an example of how Beecomender will create it's recomendation based on user's Playlists.

---

### Importing required Libraries

In [206]:
from spotipy.oauth2 import SpotifyOAuth,SpotifyClientCredentials
import spotipy.oauth2 as oauth2
import pandas as pd
import spotipy
import config
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


### Spotify's Credential Manager Authentication

In [207]:
client_credentials_manager = SpotifyClientCredentials(client_id = config.client_ID, client_secret = config.client_secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

### Getting User's Playlist data

In [208]:
def extract_playlist_URI(playlist_URL):
    # Extract the playlist URI from the playlist URL
    playlist_URI = playlist_URL.split("/")[-1].split("?")[0]
    return playlist_URI

def get_tracks_info(playlist_URL):
    # Get the playlist URI
    playlist_URI = extract_playlist_URI(playlist_URL)
    
    # Get the tracks of the playlist
    playlist_tracks = sp.playlist_tracks(playlist_URI)
    
    tracks_data = []
    for track in playlist_tracks["items"]:
        playlist_data = {}
        
        # Get the URI of the track
        track_uri = track["track"]["uri"]
        playlist_data.update({'track_uri' : track_uri})
        
        # Get the name of the track
        playlist_data.update({'track_name' : track["track"]["name"]})
        
        # Get the main artist URI
        artist_uri = track["track"]["artists"][0]["uri"]
        playlist_data.update({ 'artist_uri' : artist_uri})
        
        # Get information about the main artist
        artist_info = sp.artist(artist_uri)
        
        # Get the name, popularity, and genres of the artist
        playlist_data.update({'artist_name' : track["track"]["artists"][0]["name"]})
        playlist_data.update({ 'artist_popularity' : artist_info["popularity"]})
        playlist_data.update({ 'genres' : artist_info["genres"]})
        
        # Get the album information
        playlist_data.update({ 'album_name' : track["track"]["album"]["name"]})
        playlist_data.update({ 'album_uri' : track["track"]["album"]["uri"]})
        playlist_data.update({ 'release_date' : track["track"]["album"]["release_date"]})
        
        # Get the popularity of the track
        playlist_data.update({'track_popularity' : track["track"]["popularity"]})
        
        # Get the audio features of the track
        audio_features = sp.audio_features(track_uri)
        playlist_data.update(audio_features[0])
        
        keys_to_delete = ['type', 'id', 'uri', 'track_href', 'analysis_url']
        
        # Remove unnecessary keys from the playlist data
        for key in keys_to_delete:
            if key in playlist_data:
                del playlist_data[key]
        
        # Add the playlist data to the tracks data list
        tracks_data.append(playlist_data)

    return tracks_data
        

In [209]:
URL = r'https://open.spotify.com/playlist/3MwWplprtniJHPxuWYxRZf'
data = get_tracks_info(URL)
data = pd.DataFrame(data)
data

,track_uri,track_name,artist_uri,artist_name,artist_popularity,genres,album_name,album_uri,release_date,track_popularity,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,spotify:track:1NtIMM4N0cFa1dNzN15chl,Believer,spotify:artist:53XhwfbYqKCa1cC15pYq2q,Imagine Dragons,88,"[modern rock, pop, rock]",Evolve,spotify:album:51s30K7a36ORx6AAaPBRRY,2017-06-23,0,...,-4.288,0,0.1080,0.051200,0.000000,0.119,0.719,124.988,204347,4
1,spotify:track:4dASQiO1Eoo3RJvt74FtXB,"Sucker for Pain (with Wiz Khalifa, Imagine Dra...",spotify:artist:55Aa2cqylxrFIXC767Z865,Lil Wayne,85,"[hip hop, new orleans rap, pop rap, rap, trap]",Sucker for Pain (with Logic & Ty Dolla $ign fe...,spotify:album:704GHNtZhEe9TBgleCNNGv,2016-06-24,77,...,-4.378,0,0.3170,0.255000,0.000000,0.650,0.739,169.021,243491,4
2,spotify:track:0WpRF4X2LyMTZnqqrXX19U,Get Jinxed,spotify:artist:47mIJdHORyRerp4os813jD,League of Legends,77,"[speedrun, video game music]",The Music of League of Legends Vol. 1,spotify:album:4pg51OJaG6MGlpRFxWWLpr,2015-01-28,66,...,-7.128,0,0.0930,0.000728,0.004580,0.105,0.191,180.029,154960,4
3,spotify:track:5VnDkUNyX6u5Sk0yZiP8XB,Thunder,spotify:artist:53XhwfbYqKCa1cC15pYq2q,Imagine Dragons,88,"[modern rock, pop, rock]",Evolve,spotify:album:51s30K7a36ORx6AAaPBRRY,2017-06-23,0,...,-4.804,1,0.0434,0.007820,0.150000,0.154,0.241,167.985,187147,4
4,spotify:track:1Ntj0hZfncXCQ5hij7igIE,Demons,spotify:artist:53XhwfbYqKCa1cC15pYq2q,Imagine Dragons,88,"[modern rock, pop, rock]",Night Visions (Deluxe),spotify:album:1rzDtYMpZDhRgKNigB467r,2012-09-04,70,...,-3.015,1,0.0321,0.190000,0.000250,0.329,0.428,89.938,175200,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,spotify:track:6uBhi9gBXWjanegOb2Phh0,Stay,spotify:artist:2qxJFvFYMEDqd7ui6kSAcq,Zedd,75,"[complextro, edm, german techno, pop, pop dance]",Stay,spotify:album:1I4W7JKzYbl8VKRfD61DIS,2017-02-23,77,...,-5.025,0,0.0622,0.253000,0.000000,0.116,0.544,102.040,210091,4
96,spotify:track:78SAVldsO3Xzm2R2Ko6E66,Sirens (feat. Nevve),spotify:artist:3fjs4zbBFxEFFe8Wyojo0G,Elephante,53,"[pop dance, pop edm]",I Am The Elephante,spotify:album:3VNIaZWTV7gsdX721ul0Zb,2016-09-14,0,...,-4.698,0,0.0471,0.009940,0.000002,0.383,0.203,101.976,249412,4
97,spotify:track:6aiHF2IbJr6lG4Vu9em8KF,Believer - Kaskade Remix,spotify:artist:53XhwfbYqKCa1cC15pYq2q,Imagine Dragons,88,"[modern rock, pop, rock]",Believer (Kaskade Remix),spotify:album:2wVV49qXJai8kmCb1Czp0p,2017-06-09,62,...,-5.084,0,0.0509,0.181000,0.000002,0.118,0.182,126.022,190893,4
98,spotify:track:7EHosyNV4o7Jml6GV3agZF,House of Mirrors,spotify:artist:49CE2ffZ6Z3zeYSDauSKck,Tungevaag,67,"[big room, nordic house]",Russian Roulette,spotify:album:2yEQWQCLmcDcWJJ6m95ufZ,2015-11-27,20,...,-5.555,0,0.0317,0.000590,0.000457,0.134,0.460,127.997,208130,4


### Preprocessing the Data to 'vectorize' it.
Once we have the data from User's playlist, we need to preprocess it. To make the recomendations, we'll summarize the whole playlist as a single song.

**Dropping the unnecesary columns**

In [210]:
dropped_df = data.drop(['artist_name', 'artist_uri', 'track_name', 'album_uri', 'album_name'], axis = 1)
dropped_df

,track_uri,artist_popularity,genres,release_date,track_popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,spotify:track:1NtIMM4N0cFa1dNzN15chl,88,"[modern rock, pop, rock]",2017-06-23,0,0.772,0.780,10,-4.288,0,0.1080,0.051200,0.000000,0.119,0.719,124.988,204347,4
1,spotify:track:4dASQiO1Eoo3RJvt74FtXB,85,"[hip hop, new orleans rap, pop rap, rap, trap]",2016-06-24,77,0.502,0.786,9,-4.378,0,0.3170,0.255000,0.000000,0.650,0.739,169.021,243491,4
2,spotify:track:0WpRF4X2LyMTZnqqrXX19U,77,"[speedrun, video game music]",2015-01-28,66,0.434,0.884,7,-7.128,0,0.0930,0.000728,0.004580,0.105,0.191,180.029,154960,4
3,spotify:track:5VnDkUNyX6u5Sk0yZiP8XB,88,"[modern rock, pop, rock]",2017-06-23,0,0.604,0.826,0,-4.804,1,0.0434,0.007820,0.150000,0.154,0.241,167.985,187147,4
4,spotify:track:1Ntj0hZfncXCQ5hij7igIE,88,"[modern rock, pop, rock]",2012-09-04,70,0.505,0.710,3,-3.015,1,0.0321,0.190000,0.000250,0.329,0.428,89.938,175200,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,spotify:track:6uBhi9gBXWjanegOb2Phh0,75,"[complextro, edm, german techno, pop, pop dance]",2017-02-23,77,0.690,0.622,5,-5.025,0,0.0622,0.253000,0.000000,0.116,0.544,102.040,210091,4
96,spotify:track:78SAVldsO3Xzm2R2Ko6E66,53,"[pop dance, pop edm]",2016-09-14,0,0.621,0.783,6,-4.698,0,0.0471,0.009940,0.000002,0.383,0.203,101.976,249412,4
97,spotify:track:6aiHF2IbJr6lG4Vu9em8KF,88,"[modern rock, pop, rock]",2017-06-09,62,0.730,0.634,10,-5.084,0,0.0509,0.181000,0.000002,0.118,0.182,126.022,190893,4
98,spotify:track:7EHosyNV4o7Jml6GV3agZF,67,"[big room, nordic house]",2015-11-27,20,0.783,0.820,5,-5.555,0,0.0317,0.000590,0.000457,0.134,0.460,127.997,208130,4


In [211]:
def vectorize_genres(data_df):
    # Create a copy of the input dataframe
    df = data_df.copy()

    # Create an instance of TfidfVectorizer
    tfidf = TfidfVectorizer()

    # Apply TF-IDF vectorization on the 'genres' column
    tfidf_matrix = tfidf.fit_transform(df['genres'].apply(lambda x: " ".join(x)))

    # Convert the TF-IDF matrix to a DataFrame
    genre_df = pd.DataFrame(tfidf_matrix.toarray())

    # Set column names for the genre DataFrame
    genre_df.columns = ['genre' + "|" + i for i in tfidf.get_feature_names_out()]

    # Reset the index of the genre DataFrame
    genre_df.reset_index(drop=True, inplace=True)

    # Concatenate the original dataframe and the genre DataFrame horizontally
    final_df = pd.concat([df, genre_df], axis=1)
    
    #Drop the original genres column.
    final_df = final_df.drop('genres', axis = 1)
    
    # Return the final dataframe
    return final_df

In [212]:
preprocess_data = vectorize_genres(dropped_df)
preprocess_data

,track_uri,artist_popularity,release_date,track_popularity,danceability,energy,key,loudness,mode,speechiness,...,genre|spanish,genre|speedrun,genre|synthpop,genre|techno,genre|trap,genre|tropical,genre|uk,genre|video,genre|wave,genre|yacht
0,spotify:track:1NtIMM4N0cFa1dNzN15chl,88,2017-06-23,0,0.772,0.780,10,-4.288,0,0.1080,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0
1,spotify:track:4dASQiO1Eoo3RJvt74FtXB,85,2016-06-24,77,0.502,0.786,9,-4.378,0,0.3170,...,0.0,0.000000,0.0,0.000000,0.275583,0.0,0.0,0.000000,0.0,0.0
2,spotify:track:0WpRF4X2LyMTZnqqrXX19U,77,2015-01-28,66,0.434,0.884,7,-7.128,0,0.0930,...,0.0,0.533303,0.0,0.000000,0.000000,0.0,0.0,0.533303,0.0,0.0
3,spotify:track:5VnDkUNyX6u5Sk0yZiP8XB,88,2017-06-23,0,0.604,0.826,0,-4.804,1,0.0434,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0
4,spotify:track:1Ntj0hZfncXCQ5hij7igIE,88,2012-09-04,70,0.505,0.710,3,-3.015,1,0.0321,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,spotify:track:6uBhi9gBXWjanegOb2Phh0,75,2017-02-23,77,0.690,0.622,5,-5.025,0,0.0622,...,0.0,0.000000,0.0,0.496983,0.000000,0.0,0.0,0.000000,0.0,0.0
96,spotify:track:78SAVldsO3Xzm2R2Ko6E66,53,2016-09-14,0,0.621,0.783,6,-4.698,0,0.0471,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0
97,spotify:track:6aiHF2IbJr6lG4Vu9em8KF,88,2017-06-09,62,0.730,0.634,10,-5.084,0,0.0509,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0
98,spotify:track:7EHosyNV4o7Jml6GV3agZF,67,2015-11-27,20,0.783,0.820,5,-5.555,0,0.0317,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0


In [213]:
def extract_decade(date_str):
    year = int(date_str.split('-')[0])
    decade = (year // 10) * 10
    return int(decade)

def preprocess_decade(data_df):
    # Make a copy of the original DataFrame
    df = data_df.copy()
    
    # Apply the function to the "release_date" column
    df['release_date'] = df['release_date'].apply(extract_decade)
    
    return df

In [214]:
# Get the preprocessed release Date
preprocess_data = preprocess_decade(preprocess_data)
preprocess_data['release_date']

0     2010
1     2010
2     2010
3     2010
4     2010
      ... 
95    2010
96    2010
97    2010
98    2010
99    2010
Name: release_date, Length: 100, dtype: int64

In [215]:
def create_summarized_data(playlist_data):
    # Calculate the mean of each column
    summarized_data = playlist_data.mean(numeric_only = 'True').to_frame(name='mean_value')
    
    return summarized_data

In [216]:
summarized_data = create_summarized_data(preprocess_data).T
summarized_data

,artist_popularity,release_date,track_popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,...,genre|spanish,genre|speedrun,genre|synthpop,genre|techno,genre|trap,genre|tropical,genre|uk,genre|video,genre|wave,genre|yacht
mean_value,73.46,2007.0,51.45,0.57261,0.7679,5.37,-5.45478,0.47,0.077968,0.080697,...,0.019949,0.005333,0.002307,0.00497,0.008153,0.023,0.015172,0.005333,0.010572,0.007941


# Create the recomendations
Once we have the summarized playlist data, we'll use the metric of Cosine Similarity included in sklearn.metrics library. This will give us a parameter on how similar is the playlist to every song in the dataset.

---

**We load the whole database to generate the recommendations**

In [217]:
tracks_database = pd.read_csv(r'Data\model_data.csv')
tracks_database

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,genre|ye,genre|yodeling,genre|york,genre|youth,genre|zambian,genre|zhongguo,genre|zilizopendwa,genre|zolo,genre|zouk,genre|zuliana
0,0.904,0.813,4,-7.105,0,0.1210,0.0311,0.006970,0.0471,0.810,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.884,0.677,1,-5.603,1,0.2830,0.0778,0.000000,0.0732,0.584,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.794,0.805,0,-6.554,1,0.2300,0.5380,0.122000,0.0952,0.658,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.622,0.669,9,-8.419,1,0.3290,0.0266,0.000003,0.1520,0.570,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.797,0.750,0,-9.369,1,0.2470,0.5330,0.108000,0.0950,0.740,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34316,0.434,0.279,8,-11.947,1,0.0465,0.7700,0.042400,0.1330,0.157,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34317,0.416,0.394,11,-9.269,1,0.0641,0.5130,0.001550,0.0988,0.131,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34318,0.669,0.228,2,-12.119,1,0.0690,0.7920,0.065000,0.0944,0.402,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34319,0.493,0.727,1,-5.031,1,0.2170,0.0873,0.000000,0.1290,0.289,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [218]:
def align_dataframes(playlist_database, tracks_database):
    #Set a copy of the dataframes
    playlist_data = playlist_database.copy()
    tracks_data = tracks_database.copy()
    
    # Find missing columns in playlist_data
    missing_columns = list(set(tracks_data.columns) - set(playlist_data.columns))

    # Create a DataFrame with missing columns and default value of zero
    missing_columns_df = pd.DataFrame(0, index=playlist_data.index, columns=missing_columns)

    # Concatenate missing_columns_df with playlist_data
    playlist_data = pd.concat([playlist_data, missing_columns_df], axis=1)

    # Find missing columns in tracks_database
    missing_columns = list(set(playlist_data.columns) - set(tracks_data.columns))

    # Create a DataFrame with missing columns and default value of zero
    missing_columns_df = pd.DataFrame(0, index=tracks_data.index, columns=missing_columns)

    # Concatenate missing_columns_df with tracks_database
    tracks_data = pd.concat([tracks_data, missing_columns_df], axis=1)

    return playlist_data, tracks_data


In [219]:
# Assuming you have the playlist_data and tracks_database DataFrames
aligned_playlist_data, aligned_tracks_database = align_dataframes(summarized_data, tracks_database)


In [220]:
aligned_playlist_data

,artist_popularity,release_date,track_popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,...,genre|vegan,genre|fuzzpop,genre|brit,genre|ontario,genre|bansuri,genre|korean,genre|krishna,genre|brooklyn,genre|jacksonville,genre|solipsynthm
mean_value,73.46,2007.0,51.45,0.57261,0.7679,5.37,-5.45478,0.47,0.077968,0.080697,...,0,0,0,0,0,0,0,0,0,0


In [221]:
aligned_tracks_database

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,genre|yodeling,genre|york,genre|youth,genre|zambian,genre|zhongguo,genre|zilizopendwa,genre|zolo,genre|zouk,genre|zuliana,genre|medieval
0,0.904,0.813,4,-7.105,0,0.1210,0.0311,0.006970,0.0471,0.810,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.884,0.677,1,-5.603,1,0.2830,0.0778,0.000000,0.0732,0.584,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0.794,0.805,0,-6.554,1,0.2300,0.5380,0.122000,0.0952,0.658,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0.622,0.669,9,-8.419,1,0.3290,0.0266,0.000003,0.1520,0.570,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.797,0.750,0,-9.369,1,0.2470,0.5330,0.108000,0.0950,0.740,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34316,0.434,0.279,8,-11.947,1,0.0465,0.7700,0.042400,0.1330,0.157,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
34317,0.416,0.394,11,-9.269,1,0.0641,0.5130,0.001550,0.0988,0.131,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
34318,0.669,0.228,2,-12.119,1,0.0690,0.7920,0.065000,0.0944,0.402,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
34319,0.493,0.727,1,-5.031,1,0.2170,0.0873,0.000000,0.1290,0.289,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [222]:
def generate_recommendations(playlist_data, tracks_database, recommendation_num=30):
    tracks_data = tracks_database.copy()
    
    #Get only the nummeric features
    playlist_vector = playlist_data.drop('track_uri', axis=1).values
    track_vectors = tracks_data.drop('track_uri', axis=1).values
    
    #Normalize the features before making the recommendations
    scaler = MinMaxScaler()
    track_vectors = scaler.fit_transform(track_vectors)
    playlist_vector = scaler.transform(playlist_vector)

    # Calculate cosine similarity between the playlist and the complete song set
    similarity_scores = cosine_similarity(track_vectors, playlist_vector)[:, 0]
    tracks_data['similarity'] = similarity_scores

    top_recommendations = tracks_data.sort_values('similarity', ascending=False).head(recommendation_num)
    return top_recommendations.index.to_list(), tracks_data['similarity']

def get_recommendations_info(recommendations_id, tracks_database):
    # Filter 'tracks_database' DataFrame based on indices from 'recommendations_id'
    track_uris = tracks_database.loc[recommendations_id, 'track_uri']
    
    # Initialize lists to store the track information
    track_names = []
    track_artists = []
    track_albums = []
    track_urls = []
    
    # Retrieve track information for each URI
    for track_uri in track_uris:
        track_info = sp.track(track_uri)

        # Extract the desired information
        name = track_info['name']
        artists = ', '.join([artist['name'] for artist in track_info['artists']])
        album = track_info['album']['name']
        url = track_info['external_urls']['spotify']

        # Append the information to the respective lists
        track_names.append(name)
        track_artists.append(artists)
        track_albums.append(album)
        track_urls.append(url)

    # Create a DataFrame to store the track information
    track_info_df = pd.DataFrame({
        'Name': track_names,
        'Artist': track_artists,
        'Album': track_albums,
        'URL': track_urls
    })

    return track_info_df

In [223]:
recommendations, similarity = generate_recommendations(aligned_playlist_data, aligned_tracks_database)

In [224]:
similarity

0        0.403740
1        0.368275
2        0.313670
3        0.328050
4        0.366831
           ...   
34316    0.171618
34317    0.119235
34318    0.139617
34319    0.326775
34320    0.145600
Name: similarity, Length: 34321, dtype: float64

In [227]:
recommendations

[7699,
 13290,
 8159,
 14281,
 26513,
 12472,
 31846,
 13948,
 28842,
 5208,
 12439,
 14253,
 18536,
 19042,
 33228,
 8066,
 2163,
 5324,
 17598,
 8060,
 14289,
 2159,
 23060,
 8184,
 8105,
 30848,
 12046,
 18449,
 26982,
 19944]

In [225]:
recommendations

'''
[7699,
 13290,
 8159,
 14281,
 26513,
 12472,
 31846,
 13948,
 28842,
 12439,
 5208,
 14253,
 18536,
 19042,
 33228,
 8066,
 2163,
 5324,
 17598,
 8060,
 14289,
 2159,
 23060,
 8184,
 8105,
 18449,
 12046,
 30848,
 2167,
 26982]
'''

'\n[7699,\n 13290,\n 8159,\n 14281,\n 26513,\n 12472,\n 31846,\n 13948,\n 28842,\n 12439,\n 5208,\n 14253,\n 18536,\n 19042,\n 33228,\n 8066,\n 2163,\n 5324,\n 17598,\n 8060,\n 14289,\n 2159,\n 23060,\n 8184,\n 8105,\n 18449,\n 12046,\n 30848,\n 2167,\n 26982]\n'

In [226]:
recommendations_info = get_recommendations_info(recommendations, tracks_database)
recommendations_info

,Name,Artist,Album,URL
0,Bring Me To Life,Evanescence,Fallen,https://open.spotify.com/track/0COqiPhxzoWICwF...
1,Can't Be Touched (feat. Mr. Magic & Trouble),"Roy Jones Jr., Mr Magic, Trouble","Body Head Bangerz, Vol. 1",https://open.spotify.com/track/3zmduBNsQ6BPDTZ...
2,Duality,Slipknot,Vol. 3: The Subliminal Verses,https://open.spotify.com/track/61mWefnWQOLf90g...
3,Highway to Hell,AC/DC,Highway to Hell,https://open.spotify.com/track/2zYzyRzz6pRmhPz...
4,Sarcasm,Get Scared,Best Kind Of Mess,https://open.spotify.com/track/40i81AcCCxiVEFU...
5,In the End,Linkin Park,Hybrid Theory (Bonus Edition),https://open.spotify.com/track/60a0Rd6pjrkxjPb...
6,La Planta,Caos,La Vida Gacha,https://open.spotify.com/track/2GggG2lQVYuus2a...
7,It's My Life,Bon Jovi,Crush,https://open.spotify.com/track/0v1XpBHnsbkCn7i...
8,Whisper,Burn The Ballroom,Melodies for the Outsiders,https://open.spotify.com/track/4PSnOwlsG9tGGvy...
9,Best of You,Foo Fighters,In Your Honor,https://open.spotify.com/track/5FZxsHWIvUsmSK1...
